In [24]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

In [64]:
df = pd.read_csv('data/data.csv', index_col=0).set_index(['equity','date'])

In [26]:
df_prices = df['PX_LAST'].unstack(level=0).replace(0, np.nan)
df_log_rtn = df_prices.apply(lambda x: np.log(x / x.shift(1))).dropna(how='all')
benchmark_rtn = df_log_rtn.mean(skipna=True, axis='columns')

In [27]:
def select_best_equities_from_factor(df_full, df_rnt, factor_selected, num_equities):
    factor_values = df_full[factor_selected].unstack(level=0).iloc[1:]
    equities =  factor_values.apply(
        lambda fact_val: fact_val.loc[~df_rnt.loc[fact_val.name, fact_val.index].isna()
        ].sort_values(ascending=False).head(num_equities).index.values,
        axis='columns', result_type='expand')
    equities.columns = [str(i) + "_name" for i in range(len(equities.columns))]
    return equities, factor_values

def compute_return_from_equities(df_rtn, equities):
    equities_rtn = equities.apply(lambda row: df_rtn.loc[row.name, row.values].values, axis='columns', result_type='expand')
    equities_rtn.columns = [str(i) + "_rtn" for i in range(len(equities_rtn.columns))]
    equities_rtn.columns.name = 'equity_rtn'
    return equities_rtn

def extract_portfolio_state(equities, factor_values):
    equities_sparse = pd.DataFrame(index=factor_values.index, columns=factor_values.columns, data=0)
    for date in equities_sparse.index:
        equities_sparse.loc[date, equities.loc[date].values] = 1
    equities_sparse = (2 * equities_sparse - equities_sparse.shift(1, fill_value=0)).dropna()
    return equities_sparse.applymap(lambda x: 'not in' if x == 0 else 'in' if x == 1 else 'added' if x == 2 else 'removed')

def apply_fees(equities_rtn, equities_names, port_state, fees):
    return equities_rtn.apply(
        lambda rtn_row: rtn_row -
                        (fees * (port_state.loc[rtn_row.name, equities_names.loc[rtn_row.name]] == 'added')
                         * (1 + rtn_row).values).values,
        axis='columns')

def compute_univariate_strategy(df_full, df_rtn, factor_selected, num_equities, market_rtn, fees=None):
    equities_names, factor_values = select_best_equities_from_factor(df_full, df_rtn, factor_selected, num_equities)
    equities_rtn = compute_return_from_equities(df_rtn, equities_names)
    port_state = extract_portfolio_state(equities_names, factor_values)

    if fees is not None:
        equities_rtn = apply_fees(equities_rtn, equities_names, port_state, fees)

    equities = equities_names.merge(equities_rtn, left_index=True, right_index=True)
    equities = equities.iloc[:, np.concatenate([[i, i + num_equities] for i in range(num_equities)])]
    equities.columns = pd.MultiIndex.from_product([range(num_equities), ['name', 'rtn']])
    equities.columns.names = ['equity_pos', 'info']

    ris = equities.apply(lambda row: np.average(row.loc[:, "rtn"].values), axis='columns').to_frame(name='strategy_rtn')
    ris['strategy_alpha'] = ris['strategy_rtn'] - market_rtn
    info_ratio = ris['strategy_alpha'].mean() / ris['strategy_alpha'].std()
    return equities, ris, info_ratio, port_state

In [28]:
def compute_zscore_strategy(df_full, df_rtn, factors_selected, num_equities, market_rtn, fees=None):
    df_full['zscore'] = df_full[factors_selected].groupby(level=0).apply(lambda x: zscore(x).mean(axis=1)).values

    return compute_univariate_strategy(df_full, df_rtn, 'zscore', num_equities, market_rtn, fees)

In [77]:
def compute_sequential_screening(df_full, df_rtn, filter_factor, filter_n_equities, market_rtn, fees=None):
    df_filtered = df_full.reset_index()
    for n_equities, factor in zip(filter_n_equities[:-1], filter_factor[:-1]):
        eq_filter = select_best_equities_from_factor(df_filtered.set_index(['equity', 'date']), df_rtn, factor, n_equities)[0]
        df_filtered = pd.concat([df_filtered.loc[(df_filtered['date'] == date) & (df_filtered['equity'].isin(equities.values))
                                 ] for date, equities in eq_filter.iterrows()])
    return compute_univariate_strategy(df_filtered.set_index(['equity', 'date']), df_rtn, filter_factor[-1], filter_n_equities[-1], market_rtn, fees)

# Applicazione delle strategie univariate su più fattori

In [30]:
# Calcolo equities e information ratio per ogni factor scelto
testing_factors = ['PE_RATIO', 'EBITDA_MARGIN', 'PX_TO_BOOK_RATIO', 'NORMALIZED_ACCRUALS_CF_METHOD', 'RSI_14D', 'VOLATILITY_30D', 'CUR_MKT_CAP', 'OPERATING_ROIC']

strs_ratio = pd.Series(index=testing_factors, name='info_ratio', data=[compute_univariate_strategy(df, df_log_rtn, factor, 10, benchmark_rtn,
                                                                                                    fees=0.001)[2]
                                                                       for factor in testing_factors])

best_factors = strs_ratio.sort_values(ascending=False).head(4).index.values
best_factors

array(['RSI_14D', 'PX_TO_BOOK_RATIO', 'VOLATILITY_30D', 'OPERATING_ROIC'],
      dtype=object)

In [22]:
zscore_strategy = compute_zscore_strategy(df, df_log_rtn, best_factors, 10, benchmark_rtn, fees=0.001)

# Sequential screening

In [78]:
seq_strategy = compute_sequential_screening(df, df_log_rtn, best_factors, [400, 200, 100, 30], benchmark_rtn, fees=0.001)
